In [1]:
import pandas as pd
import numpy as np
import backtrader as bt
import matplotlib.pyplot as plt
import pyfolio as pf
import quantstats as qs
import mplfinance as mpf
import pyarrow as pa
import pyarrow.parquet as pq
import talib as ta
import importlib

/Users/Jason/work/source/03_ThorpAI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
import sys
sys.path.append('/Users/Jason/work/source/03_ThorpAI')

In [12]:
from btc_model.core.update_data.okx_downloader import OKxDownloader

from btc_model.setting.setting import get_settings
from btc_model.core.common.const import PROJECT_NAME, Interval
from btc_model.core.util.file_util import FileUtil


from btc_model.core.common.const import PROJECT_NAME
from btc_model.core.common.const import (Exchange,
                                         Interval,
                                         InstrumentType,
                                         Product,
                                         ProviderType,
                                         EntityType
                                         )
from btc_model.core.common.context import Context
from btc_model.core.update_data.okx_downloader import OKxDownloader
from btc_model.core.update_data.binance_downloader import BinanceDownloader
from btc_model.core.data_loader.crypto_data_loader import CryptoDataLoader
from btc_model.core.util.log_util import Logger
from btc_model.core.util.file_util import FileUtil
from btc_model.setting.setting import get_settings

from btc_model.indicator.base_indicator import BaseIndicator
from btc_model.indicator.indicator_bollinger import IndicatorBollinger
from btc_model.indicator.indicator_macd import IndicatorMACD
from btc_model.indicator.indicator_mayer_multiple import IndicatorMayerMultiple
from btc_model.indicator.indicator_pi_cycle import IndicatorPiCycle
from btc_model.indicator.indicator_rsi import IndicatorRSI


In [10]:
data_directory = FileUtil.get_project_dir(project_name=PROJECT_NAME, sub_dir='data')

In [ ]:
# 更新了 CryptoDataLoader 的代码后，重新加载模块
import btc_model.core.data_loader.crypto_data_loader
importlib.reload(btc_model.core.data_loader.crypto_data_loader)

In [13]:
data_loader = CryptoDataLoader(data_directory)
df_instruments = data_loader.load_instruments(exchange=Exchange.OKX)
df_indicator = data_loader.load_indicator_data(exchange=Exchange.OKX, interval=Interval.DAILY, provider_type=ProviderType.OKX)
df_indicator = df_indicator[df_indicator['symbol_id'] == 'BTC-USDT']

In [ ]:
df_indicator

In [ ]:
import san